In [ ]:
import cv2
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
import time

In [ ]:
# 1️⃣ 모델 정의 (이전과 동일)
class DrivingNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.resnet18(weights=None)
        num_ftrs = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(num_ftrs, 2)  # steer, speed

    def forward(self, x):
        return self.backbone(x)

In [ ]:
# 2️⃣ 전처리 transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
# 3️⃣ 학습된 모델 불러오기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DrivingNet().to(device)
model.load_state_dict(torch.load("driving_model.pth", map_location=device))
model.eval()

In [ ]:
# 4️⃣ 카메라 입력 (0번 장치 또는 영상 파일)
cap = cv2.VideoCapture(0)  # or 'drive_test.mp4'

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- 추론 ---
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(tensor)[0]
        steer, speed = output.cpu().numpy()

    # --- 결과 시각화 ---
    steer_display = np.clip(steer, -1, 1)
    speed_display = max(0, speed)

    text = f"Steer: {steer_display:.3f}  |  Speed: {speed_display:.2f}"
    cv2.putText(frame, text, (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    # 조향 방향 표시 (예: steer > 0 → 오른쪽)
    center = (frame.shape[1]//2, frame.shape[0]-50)
    steer_line = (int(center[0] + steer_display * 100), center[1]-50)
    cv2.arrowedLine(frame, center, steer_line, (255,0,0), 5)

    cv2.imshow("NSS: Neural Steering System", frame)

    # 차량 제어로 송신 (예시)
    # send_to_vehicle(steer, speed)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()